In [ ]:
import openpyxl
import re
import os
import pm4py

from Declare4Py.Encodings.Aggregate import Aggregate
from Declare4Py.Encodings.IndexBased import IndexBased
from Declare4Py.Encodings.Static import Static
from Declare4Py.Encodings.PreviousState import PreviousState
from Declare4Py.Encodings.LastState import LastState
from Declare4Py.Encodings.Ngram import Ngram
from Declare4Py.Encodings.Declare import Declare

# Assuming that AccuracyScore class and preprocessing function are defined somewhere
import pandas as pd
import numpy as np
import re
import pickle
import time
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import permutations
import json, os
from collections import OrderedDict
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

from Declare4Py.Encodings.Aggregate import Aggregate
import re

In [ ]:
org_path = os.getcwd()
input_path = os.sep.join([str(org_path), "input"])
output_path = os.sep.join([str(org_path), "output"])
train_temp = os.sep.join([str(org_path), "train_temp"])
train_path = os.sep.join([str(org_path), "train"])
temp_path = os.sep.join([str(org_path), "temp"])
result_path = os.sep.join([str(org_path), "result"])
concat_path = os.sep.join([str(org_path), "concat"])

In [ ]:
# clean concat

file_list = os.listdir(temp_path)

for i in range(0, 1):
    csv_list = ['prefix' + str(i) + '_bpic2012_O_ACCEPTED-COMPLETE_clean.csv' for i in range(1, 31)] # change (1)

    # csv_list를 prefix1부터 prefix30까지 정렬하기
    csv_list = sorted(csv_list, key=lambda x: int(x.split('prefix')[1].split('_')[0]))

    encoder_df = pd.DataFrame()  # 빈 데이터프레임 생성

    for k in range(0, len(csv_list)):
        df = pd.read_csv(temp_path + '/' + csv_list[k])

        # Timestamp를 기준으로 앞 쪽 80%를 train 데이터로 선택
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df_sorted = df.sort_values(by='Timestamp')
        train_length = int(len(df_sorted) * 0.8)
        train_data = df_sorted.iloc[:train_length]

        # 인코딩 수행
        encoder = Aggregate(case_id_col=['Case'], cat_cols=['Activity', 'label'], boolean=False)
        encoder_df_temp = encoder.fit_transform(df)
        encoder_df_temp['label'] = encoder_df_temp['label_deviant'].apply(lambda x: 0 if x > 0 else 1)
        encoder_df_temp.drop(['label_deviant', 'label_regular'], axis=1, inplace=True)

        # train 데이터와 test 데이터 분리
        encoder_df_temp['set'] = np.where(encoder_df_temp.index.isin(train_data['Case']), 'train', 'test')
        train = encoder_df_temp[encoder_df_temp['set'] == 'train']
        train = train.drop(['set'], axis=1)

        # train 데이터를 이어붙이기
        encoder_df = pd.concat([encoder_df, train])

        encoder_df = encoder_df.fillna(0)
        encoder_df = encoder_df.astype(int)
    
        # 결과 저장
        encoder_df.to_csv(concat_path + "\\" + 'clean_ACCEPTED_concat.csv', index=True) # change (2)

In [ ]:
# noise concat

file_list = os.listdir(temp_path)

noise_type = ['insert0.003', 'insert0.006', 'insert0.009',
            'moved0.003', 'moved0.006', 'moved0.009',
            'replace0.003', 'replace0.006', 'replace0.009',
            'rework0.003', 'rework0.006', 'rework0.009',
            'skip0.003', 'skip0.006', 'skip0.009']

for i in range(0, len(noise_type)):
    csv_list = [s for s in file_list if 'ACCEPTED' in s and noise_type[i] in s and s.endswith('.csv')] # change (1)

    # csv_list를 prefix1부터 prefix30까지 정렬하기
    csv_list = sorted(csv_list, key=lambda x: int(x.split('prefix')[1].split('_')[0]))

    encoder_df = pd.DataFrame()  # 빈 데이터프레임 생성

    for k in range(0, len(csv_list)):
        df = pd.read_csv(temp_path + '/' + csv_list[k])

        # Timestamp를 기준으로 앞 쪽 80%를 train 데이터로 선택
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df_sorted = df.sort_values(by='Timestamp')
        train_length = int(len(df_sorted) * 0.8)
        train_data = df_sorted.iloc[:train_length]

        # 인코딩 수행
        encoder = Aggregate(case_id_col=['Case'], cat_cols=['Activity', 'label'], boolean=False)
        encoder_df_temp = encoder.fit_transform(df)
        encoder_df_temp['label'] = encoder_df_temp['label_deviant'].apply(lambda x: 0 if x > 0 else 1)
        encoder_df_temp.drop(['label_deviant', 'label_regular'], axis=1, inplace=True)

        # train 데이터와 test 데이터 분리
        encoder_df_temp['set'] = np.where(encoder_df_temp.index.isin(train_data['Case']), 'train', 'test')
        train = encoder_df_temp[encoder_df_temp['set'] == 'train']
        train = train.drop(['set'], axis=1)

        # train 데이터를 이어붙이기
        encoder_df = pd.concat([encoder_df, train])

        encoder_df = encoder_df.fillna(0)
        encoder_df = encoder_df.astype(int)
    
        # 결과 저장
        encoder_df.to_csv(concat_path + "\\" + noise_type[i] +'_ACCEPTED_concat.csv', index=True) # change (2)